In [ ]:
import torchvision
import os
import glob
import random
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import scipy.io as scp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import skimage.io as skio
import scipy.io as scp
from torch.utils.data import Dataset, DataLoader, Subset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        return x

In [ ]:
class MyCNN(nn.Module):
  def __init__(self, num_channels, num_out_ch, img_w, img_h, num_classes):
    super(MyCNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=num_out_ch[0],
                           kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.bn1 = nn.BatchNorm2d(num_out_ch[0])
    self.conv2 = nn.Conv2d(in_channels=num_out_ch[0], out_channels=num_out_ch[1],
                           kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.bn2 = nn.BatchNorm2d(num_out_ch[1])
    self.conv3 = nn.Conv2d(in_channels=num_out_ch[1], out_channels=num_out_ch[1],
                           kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.bn3 = nn.BatchNorm2d(num_out_ch[1])
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.fc = nn.Linear(in_features = int(img_w//256)*int(img_h//256)*num_out_ch[1], out_features=num_classes)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x))) #1
    x = self.pool(F.relu(self.conv2(x))) #2
    x = self.pool(F.relu(self.conv3(x))) #3
    x = self.pool(F.relu(self.conv3(x))) #4
    x = self.pool(F.relu(self.conv3(x))) #5
    x = self.pool(F.relu(self.conv3(x))) #6
    x = self.pool(F.relu(self.conv3(x))) #7
    x = self.pool(F.relu(self.conv3(x))) #8
    x = self.fc(x.reshape(x.shape[0], -1))

    return x


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleCNN, self).__init__()

        # Define the convolutional blocks
        self.block1 = ConvBlock(input_channels, 32)
        self.block2 = ConvBlock(32, 32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.block3 = ConvBlock(32, 64)
        self.block4 = ConvBlock(64, 64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.block5 = ConvBlock(64, 128)
        self.block6 = ConvBlock(128, 128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.block7 = ConvBlock(128, 256)
        self.block8 = ConvBlock(256, 256)

        self.block9 = ConvBlock(256, 512)
        self.block10 = ConvBlock(512, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.block11 = ConvBlock(512, 512)
        self.block12 = ConvBlock(512, 512)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the size of the input to the fully connected layer
        self.fc_input_size = self._get_fc_input_size(input_channels)

        # Fully connected layers
        self.fc1 = nn.Linear(self.fc_input_size, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def _get_fc_input_size(self, input_channels):
        # Create a dummy input to pass through the convolutional layers to determine the size
        dummy_input = torch.zeros(1, input_channels, 64, 64)  # Assuming input image size is 64x64
        with torch.no_grad():
            dummy_output = self.block1(dummy_input)
            dummy_output = self.block2(dummy_output)
            dummy_output = self.pool1(dummy_output)
            dummy_output = self.block3(dummy_output)
            dummy_output = self.block4(dummy_output)
            dummy_output = self.pool2(dummy_output)
            dummy_output = self.block5(dummy_output)
            dummy_output = self.block6(dummy_output)
            dummy_output = self.pool3(dummy_output)
            dummy_output = self.block7(dummy_output)
            dummy_output = self.block8(dummy_output)
            dummy_output = self.block9(dummy_output)
            dummy_output = self.block10(dummy_output)
            dummy_output = self.pool4(dummy_output)
            dummy_output = self.block11(dummy_output)
            dummy_output = self.block12(dummy_output)
            dummy_output = self.pool5(dummy_output)
        return dummy_output.view(-1).size(0)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)

        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)

        x = self.block5(x)
        x = self.block6(x)
        x = self.pool3(x)

        x = self.block7(x)
        x = self.block8(x)

        x = self.block9(x)
        x = self.block10(x)
        x = self.pool4(x)

        x = self.block11(x)
        x = self.block12(x)
        x = self.pool5(x)

        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
NUM_OUT_CH = [8, 16]
IMAGE_W = 256
IMAGE_H = 256
BATCH_SIZE = 64
NUM_EPOCHS = 12
LR = 0.0001

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

input_channels = 3
num_classes = 10
batch_size = 32
num_epochs = 10

    # Create the model
#model = SimpleCNN(input_channels, num_classes)

# model
model = MyCNN(num_channels=3, num_out_ch=NUM_OUT_CH, img_w=IMAGE_W, img_h=IMAGE_H, num_classes=102)
model = model.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr = LR)

# Loss Function
criterion = nn.CrossEntropyLoss()

In [ ]:
flower_transform = transforms.Compose([
    transforms.Resize((IMAGE_W, IMAGE_H)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

augmented_transform = transforms.Compose([
    transforms.Resize((IMAGE_W, IMAGE_H)),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip the images randomly with a probability of 0.5
    transforms.RandomRotation(15),  # Randomly rotate images in the range (-15, 15) degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Randomly change brightness and contrast
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

root = '/content/drive/MyDrive/ActualFlowers/jpg'

train_set = torchvision.datasets.Flowers102(root = root, split = 'train', transform = augmented_transform, target_transform = None, download = False)
test_set = torchvision.datasets.Flowers102(root = root, split = 'test', transform = flower_transform, target_transform = None, download = False)
validation_set = torchvision.datasets.Flowers102(root = root, split = 'val', transform = flower_transform, target_transform = None, download = False)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def check_accuracy(loader, model, num_classes=102):
    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    # Initialize the confusion matrix
    confusion_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)

    with torch.no_grad():  # Do not calculate gradients
        for x, y in loader:
            x = x.to(device)  # Move data to the device
            y = y.to(device)  # Move labels to the device
            scores = model(x)  # Compute model output
            _, predictions = scores.max(1)  # Get the predicted classes
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

            # Update confusion matrix
            for t, p in zip(y.view(-1), predictions.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    model.train()  # Set the model back to training mode
    accuracy = float(num_correct) / num_samples  # Calculate accuracy

    # Print overall accuracy
    print(f"Got {num_correct} / {num_samples} with accuracy {accuracy * 100:.2f}%")

    # Print confusion matrix or other statistics if necessary
    # For detailed analysis, you might return or further process the confusion matrix
    return accuracy, confusion_matrix

In [ ]:
def evaluate(model, loader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct_predictions / len(loader.dataset)
    model.train()  # Set the model back to training mode
    return avg_loss, accuracy

In [ ]:
best_val_loss = float('inf')
epochs_no_improve = 0
n_epochs_stop = 10
for epoch in range(NUM_EPOCHS*5):
    running_loss = 0
    with tqdm.tqdm(train_loader, unit='batch') as tepoch:
        for index, (x, y) in enumerate(tepoch):
            x = x.to(device)
            y = y.to(device)

            # Forward pass
            y_hat = model(x)
            loss = criterion(y_hat, y)
            running_loss += loss.item()

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())

    # Compute average training loss
    avg_training_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: Training loss: {running_loss:.4f}")
    if (epoch+1)%10==0:
     check_accuracy(test_loader, model)

    # Evaluate on the validation set
    if epoch >= 10:
      validation_loss, validation_accuracy = evaluate(model, validation_loader, device)
      print(f"Epoch {epoch}: Validation loss: {validation_loss:.4f}, Validation accuracy: {validation_accuracy*100:.4f}%")

      # Check for early stopping
      if validation_loss < best_val_loss:
          best_val_loss = validation_loss
          epochs_no_improve = 0
          # Save the model if validation loss improves
          torch.save(model.state_dict(), 'best_model_v1.pth')
      else:
          epochs_no_improve += 1
          print(f"No improvement in validation loss for {epochs_no_improve} epochs.")

      # Early stopping condition
      if epochs_no_improve == n_epochs_stop:
          print("Early stopping triggered")
          break

100%|██████████| 16/16 [03:52<00:00, 14.54s/batch, loss=4.65]


Epoch 0: Training loss: 74.1755


100%|██████████| 16/16 [00:51<00:00,  3.23s/batch, loss=4.61]


Epoch 1: Training loss: 74.1708


100%|██████████| 16/16 [00:52<00:00,  3.26s/batch, loss=4.63]


...

Epoch 7: Training loss: 74.1645


100%|██████████| 16/16 [00:52<00:00,  3.30s/batch, loss=4.67]


Epoch 8: Training loss: 74.1660


100%|██████████| 16/16 [00:53<00:00,  3.31s/batch, loss=4.63]


Epoch 9: Training loss: 74.1625
